In [1]:
'''
[생략] 동물보호관리시스템 홈페이지 
http://www.animal.go.kr

<url pramater>
s_date : 시작년도
e_date : 종료년도 
s_upr_cd : 시도 
pagecnt : 페이지 번호 
'''

import urllib.request as req  # url 가져오기 
from bs4 import BeautifulSoup # tag 데이터 가져오기 

'''
년도 : 2010-01-01 ~ 오늘
시도 : 서울시 , 부산시, 대구시 
페이지 : 1~10
'''

base_url = "http://www.animal.go.kr/portal_rnl/abandonment/public_list.jsp?s_date={s_date}&e_date={e_date}&s_upr_cd={city}&s_org_cd=0000000&s_up_kind_cd=&s_kind_cd=&s_name=&s_shelter_cd=&s_wrk_cd=&s_state=&s_state_hidden=&pagecnt="
crawling_data = [] # data 저장 list
city_data = {'서울' : '6110000', '부산' : '6260000', '대구':'6270000'}

In [4]:
# 클로러 함수(시작년도, 종료년도, 도시, 페이지번호) 
def crawler_func(s_date, e_date, city, pg=10): 
    # url 구성 = base_url + parameter
    url = base_url.format(s_date=s_date, e_date=e_date, city=city_data[city])
    
    # page 번호(page 수 만큼 반복)
    for p in range(1, int(pg)+1) : # pg=10 = 1 ~ 10
        # 1) 최종 url 구성
        url2 = url + str(p) 
        #print(url2)
        
        # 2) url 요청 -> html source
        res = req.urlopen(url2)
        data = res.read()
        
        # 3) html 파싱
        html = data.decode('UTF8') # charset='euc-kr'
        soup = BeautifulSoup(html, 'html.parser')
        
        # 4) tag 기반 crawling : 태그[속성='값'] > 태그[속성='값'] 
        '''
        <div class="thumb_inner02">
        <dl class="thumbnail_table01">
        '''
        gonggos = soup.select('div[class=thumb_inner02] > dl[class=thumbnail_table01]')
        
        for gonggo in gonggos : # page 당 동물 10 정보
            # <dd> 태그 추출 
            dds = gonggo.find_all('dd') # 1동물 당 7개 칼럼 
            # print(dds)
            
            ani_one = [] # 1동물 당 7개 칼럼 저장
            for dd in dds :
                re = dd.string.strip()
                #print(re) # <dd> 태그 내용 불용어 제거(엔터,공백,특수문자)
                ani_one.append(re) # [dd1, ~ dd7]
            
            # 중첩list : [[dd1, ~ dd7], ~ [dd1, ~ dd7]]
            crawling_data.append(ani_one)    

# 클로러 함수 호출 
crawler_func('2018-01-01', '2018-01-30', '부산')
print('크롤링 동물 수 =', len(crawling_data)) # 10
print(crawling_data)


크롤링 동물 수 = 0
[]


In [5]:
# Data.frame
import pandas as pd

# vector
c1=[]; c2=[]; c3=[]; c4=[]; c5=[]; c6=[]; c7=[]  
for one in crawling_data : # 100번 반복 
    c1.append(one[0]); c2.append(one[1]);c3.append(one[2])  
    c4.append(one[3]); c5.append(one[4]);c6.append(one[5])
    c7.append(one[6]);

In [6]:
# data.frame 생성
cols = ['no','date','kind','gender','place','feature','state']
crawling_df = pd.DataFrame({'no' : c1, 'date' : c2, 'kind' : c3, 'gender':c4,
             'place' : c5, 'feature' : c6, 'state' : c7},
             columns=cols)

print(crawling_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   no       0 non-null      float64
 1   date     0 non-null      float64
 2   kind     0 non-null      float64
 3   gender   0 non-null      float64
 4   place    0 non-null      float64
 5   feature  0 non-null      float64
 6   state    0 non-null      float64
dtypes: float64(7)
memory usage: 124.0 bytes
None


In [8]:
# file save

crawling_df.to_csv("C:/Users/hyebin/Desktop/Python_ML/data/crawling_df.csv",
                    index=None, encoding='utf-8')


crawling = pd.read_csv("C:/Users/hyebin/Desktop/Python_ML/data/crawling_df.csv")
print(crawling.head())

Empty DataFrame
Columns: [no, date, kind, gender, place, feature, state]
Index: []
